# Import libraries

In [2]:
import requests
import pandas as pd
import time
import ta
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Data Collection and Preprocessing

**Explanation**

*   fetch_live_data: This function fetches the current price of BTC/USDT from Binance API.
*   fetch_historical_data: This function fetches historical data for BTC/USDT for a specified interval and limit from Binance API.
*   preprocess_data: This function calculates the percentage change in the close prices to compute returns and removes any missing values.
*   update_data: This function fetches the latest price data, adds it to the DataFrame, and updates the returns.
*   fetch_and_preprocess_data: This function fetches and preprocesses historical data for multiple intervals.
*   intervals: Different time intervals '1m' for which data is collected and processed.





In [5]:
# Fetch live data for BTC/USDT
def fetch_live_data():
    url = 'https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT'
    response = requests.get(url)
    data = response.json()
    return float(data['price'])

# Fetch historical data for BTC/USDT
def fetch_historical_data(symbol, interval, limit=1000):
    url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
    return df

# Preprocess data to add returns
def preprocess_data(df):
    df['Returns'] = df['close'].pct_change()
    df.dropna(inplace=True)
    return df

# Function to update the DataFrame with new live data
def update_data(df):
    price = fetch_live_data()
    timestamp = pd.Timestamp.now()
    new_data = pd.DataFrame({'timestamp': [timestamp], 'close': [price]})
    df = pd.concat([df, new_data], ignore_index=True)
    df.set_index('timestamp', inplace=True)
    df = preprocess_data(df)
    return df

# Fetch and preprocess data for all intervals
def fetch_and_preprocess_data(symbol, intervals, limit=1000):
    data = {}
    for interval in intervals:
        df = fetch_historical_data(symbol, interval, limit)
        df = preprocess_data(df)
        data[interval] = df
    return data

# Define intervals to fetch data for
intervals = ['1m']

# Fetch and preprocess data for all intervals
data = fetch_and_preprocess_data('BTCUSDT', intervals)

# Display the first few rows of the processed data for each interval
for interval, df in data.items():
    print(f"Interval: {interval}")
    print(df.head())

Interval: 1m
                         open      high       low     close    volume  \
timestamp                                                               
2024-07-24 02:18:00  65476.84  65512.00  65476.84  65512.00  18.70643   
2024-07-24 02:19:00  65511.99  65550.22  65500.00  65550.22  13.00383   
2024-07-24 02:20:00  65550.21  65550.22  65494.36  65494.36   8.78919   
2024-07-24 02:21:00  65494.36  65496.00  65485.79  65492.01   6.79458   
2024-07-24 02:22:00  65492.00  65515.00  65492.00  65513.69   3.63409   

                      Returns  
timestamp                      
2024-07-24 02:18:00  0.000537  
2024-07-24 02:19:00  0.000583  
2024-07-24 02:20:00 -0.000852  
2024-07-24 02:21:00 -0.000036  
2024-07-24 02:22:00  0.000331  


# Implement Trading Strategies

**Explanation**

*   IndicatorCalculationAgent: This class calculates various technical indicators such as SMA, MACD, Stochastic Oscillator, and Fibonacci retracement levels and adds them to the DataFrame.
*   PredictionAgent: This class trains a RandomForestClassifier model and predicts the trading signals based on the technical indicators.
*   TradingAgent: This class executes trades based on the predictions and keeps track of the USDT and BTC balances.
*   MonitoringAgent: This class calculates the profit or loss by comparing the current USDT balance with the initial funds.


In [8]:
# Indicator Calculation Agent
class IndicatorCalculationAgent:
    @staticmethod
    def add_indicators(df):
        df = df.copy()
        df['sma_50'] = ta.trend.sma_indicator(df['close'], window=50)
        df['sma_200'] = ta.trend.sma_indicator(df['close'], window=200)
        df['macd'] = ta.trend.macd(df['close'])
        df['macd_signal'] = ta.trend.macd_signal(df['close'])
        stoch = ta.momentum.StochasticOscillator(df['high'], df['low'], df['close'], window=9, smooth_window=3)
        df['stoch_k'] = stoch.stoch()
        df['stoch_d'] = stoch.stoch_signal()
        df['fib_retracement'] = (df['close'] - df['close'].min()) / (df['close'].max() - df['close'].min())
        df.dropna(inplace=True)
        return df

# Prediction Agent
class PredictionAgent:
    def __init__(self, model, scaler, features):
        self.model = model
        self.scaler = scaler
        self.features = features

    def train_model(self, X_train, y_train):
        self.model.fit(X_train, y_train)

    def predict(self, data):
        X = data[self.features]
        X_scaled = self.scaler.transform(X)
        return self.model.predict(X_scaled)

# Trading Agent
class TradingAgent:
    def __init__(self, initial_funds=100000, name='Agent'):
        self.initial_funds = initial_funds
        self.usdt_balance = initial_funds
        self.btc_balance = 0
        self.name = name

    def execute_trade(self, prediction, price):
        if prediction == 1 and self.usdt_balance > 0:
            self.btc_balance = self.usdt_balance / price
            self.usdt_balance = 0
            print(f"[{self.name}] Buy at {price}, BTC balance: {self.btc_balance}")
        elif prediction == 0 and self.btc_balance > 0:
            self.usdt_balance = self.btc_balance * price
            self.btc_balance = 0
            print(f"[{self.name}] Sell at {price}, USDT balance: {self.usdt_balance}")

    def get_balances(self):
        return self.usdt_balance, self.btc_balance

# Monitoring Agent
class MonitoringAgent:
    def __init__(self, initial_funds):
        self.initial_funds = initial_funds

    def calculate_profit_loss(self, usdt_balance):
        profit_loss = usdt_balance - self.initial_funds
        return profit_loss



# Train and Test Strategies

**Train Machine Learning Model**

In [11]:
# Training and testing the strategy with enhanced checks

# Step 3: Train and Test Your Strategies

class StochasticFibonacciAgent:
    def __init__(self, k_period=9, d_period=6, min_data_points=100, initial_funds=100000, name='StochasticFibonacciAgent'):
        self.k_period = k_period
        self.d_period = d_period
        self.portfolio = 0
        self.cash = initial_funds
        self.initial_funds = initial_funds
        self.model = RandomForestClassifier(n_estimators=100, random_state=42)
        self.min_data_points = min_data_points
        self.trained = False
        self.last_transaction_price = None
        self.name = name

    def calculate_stochastic(self, df):
        low_min = df['close'].rolling(window=self.k_period).min()
        high_max = df['close'].rolling(window=self.k_period).max()
        stoch_k = 100 * ((df['close'] - low_min) / (high_max - low_min))
        stoch_d = stoch_k.rolling(window=self.d_period).mean()
        return stoch_k, stoch_d

    def calculate_fibonacci_levels(self, df):
        high_price = df['close'].max()
        low_price = df['close'].min()
        level_1 = high_price - (high_price - low_price) * 0.236
        level_2 = high_price - (high_price - low_price) * 0.382
        level_3 = high_price - (high_price - low_price) * 0.618
        return level_1, level_2, level_3

    def prepare_features(self, df):
        stoch_k, stoch_d = self.calculate_stochastic(df)
        level_1, level_2, level_3 = self.calculate_fibonacci_levels(df)
        features = pd.DataFrame({
            'Stoch_K': stoch_k,
            'Stoch_D': stoch_d,
            'Fib_1': level_1,
            'Fib_2': level_2,
            'Fib_3': level_3
        })
        features['Close'] = df['close']
        return features.dropna()

    def train_model(self, df):
        features = self.prepare_features(df)
        if len(features) < self.min_data_points:
            print(f"[{self.name}] Not enough data to train the model")
            return False

        features['Target'] = features['Close'].shift(-1) > features['Close']
        features['Target'] = features['Target'].astype(int)
        X = features[['Stoch_K', 'Stoch_D', 'Fib_1', 'Fib_2', 'Fib_3']]
        y = features['Target']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.model.fit(X_train, y_train)

        # Evaluate the model
        y_pred = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)

        print(f"[{self.name}] Model accuracy: {accuracy:.2f}")
        print(f"[{self.name}] Model precision: {precision:.2f}")
        print(f"[{self.name}] Model recall: {recall:.2f}")
        print(f"[{self.name}] Model F1-score: {f1:.2f}")
        print(f"[{self.name}] Classification report:\n{report}")

        self.trained = True
        return True

    def trade(self, df):
        if len(df) < self.min_data_points or not self.trained:
            print(f"[{self.name}] Not enough data to calculate indicators or model not trained")
            return

        features = self.prepare_features(df)
        X = features[['Stoch_K', 'Stoch_D', 'Fib_1', 'Fib_2', 'Fib_3']].iloc[-1].values.reshape(1, -1)
        X = pd.DataFrame(X, columns=['Stoch_K', 'Stoch_D', 'Fib_1', 'Fib_2', 'Fib_3'])  # Ensure correct feature names
        prediction = self.model.predict(X)[0]
        current_price = df['close'].iloc[-1]

        print(f"[{self.name}] Prediction: {'Buy' if prediction else 'Sell'}, Current Price: {current_price}")

        if prediction == 1 and self.cash > 0:  # Buy signal
            self.portfolio += self.cash / current_price
            self.last_transaction_price = current_price
            self.cash = 0
            print(f"[{self.name}] Buying at price {current_price}")
        elif prediction == 0 and self.portfolio > 0:  # Sell signal
            profit_loss = (current_price - self.last_transaction_price) * self.portfolio
            self.cash += self.portfolio * current_price
            self.portfolio = 0
            print(f"[{self.name}] Selling at price {current_price}")
            print(f"[{self.name}] Profit/Loss for this transaction: {profit_loss}")

    def get_portfolio_value(self, current_price):
        return self.cash + self.portfolio * current_price




**Test Strategies on Interval 1 minute**

Explanation
* Agent Initialization: Agents for indicator calculation, prediction, trading, and monitoring are initialized.
* Data Preparation: Historical data is fetched, and technical indicators are added.
* Model Training: The RandomForestClassifier model is trained using the training data, and its performance is evaluated on the test set.
* Mocked Data Loop: A loop simulates the process of fetching new data, making predictions, and executing trades.
* Performance Monitoring: The final profit or loss is calculated and displayed.

In [14]:
# Initialize and evaluate agents on 1-minute interval
initial_funds = 100000
interval = '1m'
indicator_calculation_agent = IndicatorCalculationAgent()
scaler = StandardScaler()
model = RandomForestClassifier(n_estimators=100, random_state=42)
features = ['sma_50', 'sma_200', 'macd', 'macd_signal', 'stoch_k', 'stoch_d', 'fib_retracement']
prediction_agent = PredictionAgent(model, scaler, features)
trading_agent = TradingAgent(initial_funds)
monitoring_agent = MonitoringAgent(initial_funds)

# Fetch and prepare historical data
btc_data = fetch_historical_data('BTCUSDT', interval)
btc_data = indicator_calculation_agent.add_indicators(btc_data)

# Prepare features and target
target = (btc_data['close'].shift(-1) > btc_data['close']).astype(int)
X = btc_data[features]
y = target

# Split and scale data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model and evaluate performance
prediction_agent.train_model(X_train_scaled, y_train)
y_pred = prediction_agent.model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Model accuracy on test set: {accuracy:.2f}")
print(f"Model precision on test set: {precision:.2f}")
print(f"Model recall on test set: {recall:.2f}")
print(f"Model F1-score on test set: {f1:.2f}")
print(f"Classification report on test set:\n{classification_report(y_test, y_pred)}")

# Mocked data loop (in real scenario, fetch new data every minute)
for i in range(1, len(btc_data)):
    current_data = btc_data.iloc[:i].copy()  # Use .copy() to avoid SettingWithCopyWarning
    if len(current_data) < 100:  # Ensure sufficient data for indicators
        continue
    current_data = indicator_calculation_agent.add_indicators(current_data)
    if len(current_data) == 0:
        continue
    prediction = prediction_agent.predict(current_data.iloc[-1:])
    trading_agent.execute_trade(prediction[0], current_data['close'].iloc[-1])

# Monitor performance
usdt_balance, btc_balance = trading_agent.get_balances()
profit_loss = monitoring_agent.calculate_profit_loss(usdt_balance)
print(f"Profit/Loss: {profit_loss}")

Model accuracy on test set: 0.43
Model precision on test set: 0.44
Model recall on test set: 0.52
Model F1-score on test set: 0.47
Classification report on test set:
              precision    recall  f1-score   support

           0       0.43      0.35      0.39        82
           1       0.44      0.52      0.47        79

    accuracy                           0.43       161
   macro avg       0.43      0.44      0.43       161
weighted avg       0.43      0.43      0.43       161

[Agent] Buy at 66443.46, BTC balance: 1.505039021146701
[Agent] Sell at 66456.58, USDT balance: 100019.74611195743
[Agent] Buy at 66435.01, BTC balance: 1.5055276745191646
[Agent] Sell at 66437.01, USDT balance: 100022.75716730648
[Agent] Buy at 66317.98, BTC balance: 1.5082298521050623
[Agent] Sell at 66350.0, USDT balance: 100071.05068717088
[Agent] Buy at 66350.0, BTC balance: 1.5082298521050623
[Agent] Sell at 66430.0, USDT balance: 100191.70907533928
[Agent] Buy at 66347.96, BTC balance: 1.5100947

# Backtesting Framework

**Explanation**
* calculate_performance_metrics: This function calculates total return, Sharpe ratio, and maximum drawdown for the trading agent.
* backtest: This function simulates the trading strategy on historical data and returns the final portfolio value.
* Backtesting and Metrics Calculation: The backtesting framework is applied to agents for '1m' intervals, and performance metrics are calculated and displayed.

In [17]:
# Function to calculate performance metrics
def calculate_performance_metrics(agent, data):
    final_value = agent.get_portfolio_value(data['close'].iloc[-1])
    total_return = (final_value - agent.initial_funds) / agent.initial_funds

    returns = data['Returns'].dropna()
    sharpe_ratio = (returns.mean() / returns.std()) * np.sqrt(252)

    cumulative_returns = (1 + returns).cumprod()
    peak = cumulative_returns.cummax()
    drawdown = (cumulative_returns - peak) / peak
    max_drawdown = drawdown.min()

    return total_return, sharpe_ratio, max_drawdown

# Function to backtest agent
def backtest(agent, data):
    agent.train_model(data)
    for timestamp, row in data.iterrows():
        agent.trade(data.loc[:timestamp])
    return agent.get_portfolio_value(data['close'].iloc[-1])

# Backtest each agent and calculate performance metrics
results = {}

for interval in intervals:
    agent = StochasticFibonacciAgent(name=f'Agent_{interval}')
    agent.train_model(data[interval])
    final_value = backtest(agent, data[interval])
    total_return, sharpe_ratio, max_drawdown = calculate_performance_metrics(agent, data[interval])
    results[interval] = {
        'final_value': final_value,
        'total_return': total_return,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown
    }

# Display results
for interval, metrics in results.items():
    print(f"Interval: {interval}")
    print(f"Final Portfolio Value: {metrics['final_value']}")
    print(f"Total Return: {metrics['total_return']:.2%}")
    print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
    print(f"Maximum Drawdown: {metrics['max_drawdown']:.2%}")

[Agent_1m] Model accuracy: 0.49
[Agent_1m] Model precision: 0.43
[Agent_1m] Model recall: 0.45
[Agent_1m] Model F1-score: 0.44
[Agent_1m] Classification report:
              precision    recall  f1-score   support

           0       0.54      0.52      0.53       110
           1       0.43      0.45      0.44        88

    accuracy                           0.49       198
   macro avg       0.49      0.49      0.49       198
weighted avg       0.49      0.49      0.49       198

[Agent_1m] Model accuracy: 0.49
[Agent_1m] Model precision: 0.43
[Agent_1m] Model recall: 0.45
[Agent_1m] Model F1-score: 0.44
[Agent_1m] Classification report:
              precision    recall  f1-score   support

           0       0.54      0.52      0.53       110
           1       0.43      0.45      0.44        88

    accuracy                           0.49       198
   macro avg       0.49      0.49      0.49       198
weighted avg       0.49      0.49      0.49       198

[Agent_1m] Not enough da

# Real-Time Trading Simulation

**Explanation**
* update_data_real_time: This function fetches the latest data, updates the DataFrame, and recalculates the indicators.
* update_agents: This function updates the data for '1m' intervals and makes trading decisions based on the latest data.
* Real-Time Simulation: The simulation runs in an infinite loop, continuously updating the data and making trading decisions.

In [20]:
# Ensure the IndicatorCalculationAgent is available
indicator_calculation_agent = IndicatorCalculationAgent()

# Function to update agents in real-time
def update_data_real_time(agent, data, interval):
    new_data = fetch_historical_data('BTCUSDT', interval, limit=1)
    new_data = preprocess_data(new_data)
    data = pd.concat([data, new_data])
    data = indicator_calculation_agent.add_indicators(data)
    return data

def update_agents():
    global data
    data['1m'] = update_data_real_time(agent_1m, data['1m'], '1m')

    agent_1m.trade(data['1m'])

    print(f"1m Interval Portfolio Value: {agent_1m.get_portfolio_value(data['1m']['close'].iloc[-1])}")

# Initialize agents for different intervals
agent_1m = StochasticFibonacciAgent(name='Agent_1m')

# The agents are trained with initial data
agent_1m.train_model(data['1m'])

# Start the real-time trading simulation
while True:
    update_agents()
    time.sleep(60)  # Adjust the sleep time according to the interval


[Agent_1m] Model accuracy: 0.49
[Agent_1m] Model precision: 0.43
[Agent_1m] Model recall: 0.45
[Agent_1m] Model F1-score: 0.44
[Agent_1m] Classification report:
              precision    recall  f1-score   support

           0       0.54      0.52      0.53       110
           1       0.43      0.45      0.44        88

    accuracy                           0.49       198
   macro avg       0.49      0.49      0.49       198
weighted avg       0.49      0.49      0.49       198

[Agent_1m] Prediction: Sell, Current Price: 65808.68
1m Interval Portfolio Value: 100000.0
[Agent_1m] Prediction: Sell, Current Price: 65808.68
1m Interval Portfolio Value: 100000.0
[Agent_1m] Prediction: Sell, Current Price: 65808.68
1m Interval Portfolio Value: 100000.0
[Agent_1m] Prediction: Sell, Current Price: 65808.68
1m Interval Portfolio Value: 100000.0
[Agent_1m] Not enough data to calculate indicators or model not trained
1m Interval Portfolio Value: 100000.0
[Agent_1m] Not enough data to calculat

IndexError: single positional indexer is out-of-bounds

# Analysis and Comparison

**Explanation**
* Performance Analysis: This step involves comparing the performance of the trading strategies across different intervals using metrics like total return, Sharpe ratio, and maximum drawdown.
* Documentation: Analyze and document the findings, discussing why certain strategies performed better at specific intervals and identifying areas for improvement.

In [23]:
# Analysis and Comparison of strategies from Backtesting Framework
for interval, metrics in results.items():
    print(f"Interval: {interval}")
    print(f"Final Portfolio Value: {metrics['final_value']}")
    print(f"Total Return: {metrics['total_return']:.2%}")
    print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
    print(f"Maximum Drawdown: {metrics['max_drawdown']:.2%}")
    print("-" * 50)


Interval: 1m
Final Portfolio Value: 114206.60968904907
Total Return: 14.21%
Sharpe Ratio: 0.15
Maximum Drawdown: -2.01%
--------------------------------------------------
